In [1]:
import os

In [2]:
%pwd

'/Users/rohitsriram/Documents/ML-Develop/projects/NLP project(s)/NLP-text-summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rohitsriram/Documents/ML-Develop/projects/NLP project(s)/NLP-text-summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

#Entity- return type of a function(s)
@dataclass(frozen=True)
class DataConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from text_summarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from text_summarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager: 
    def __init__ (
        self, 
        config_filepath=CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_config(self) -> DataConfig:
        config = self.config.data
        
        create_directories([config.root_dir])
        
        data_config = DataConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_config

In [8]:
import os
import urllib.request as request
import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size

In [9]:
class Data:
    def __init__(self, config: DataConfig):
        self.config = config
        
    def download_file(self): 
        if not os.path.exists(self.config.local_data_file): 
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, 
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} Downloaded. Contains the following information. \n{headers}")
        else: 
            logger.info(f"File already exists of size {get_size(Path(self.config.local_data_file))}.")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_config = config.get_data_config()
    data = Data(config=data_config)
    data.download_file()
    data.extract_zip_file()
    
except Exception as e:
    raise e

[2024-03-14 22:53:25,908: INFO: common: YAML file: config/config.yaml loaded successfully:]
[2024-03-14 22:53:25,916: INFO: common: YAML file: params.yaml loaded successfully:]
[2024-03-14 22:53:25,920: INFO: common: Created directory at: artifacts:]
[2024-03-14 22:53:25,923: INFO: common: Created directory at: artifacts/data:]
[2024-03-14 22:53:27,320: INFO: 2556540059: artifacts/data/data.zip Downloaded. Contains the following information. 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B8B0:174E:935B8F:A3C93B:65F3B826
Accept-Ranges: bytes
Date: Fri, 15 Mar 2024 02:53:26 GMT
Via: 1.1 varnish
X-Served-By: cache-pdk-kpdk1780140-PDK
X-Ca